In [21]:
import os
import numpy as np
import pandas as pd
import keras
import tensorflow as tf
import matplotlib.pyplot as plt
from keras.layers import Dense, GlobalAveragePooling2D, MaxPooling2D, Dropout
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint
from keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import confusion_matrix
import plotly.figure_factory as ff
import PIL

In [2]:
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [3]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [4]:
from keras.applications.mobilenet import MobileNet
from keras.applications.mobilenet import preprocess_input

### Loading & Transforming our Training Data to a Numpy Array

In [5]:
.X, y = [], []
for i in os.listdir('./dataset/training_set'):
    true_class = i
    for j in os.listdir('./dataset/training_set/' + i):
        img = image.load_img('./dataset/training_set/' + i + '/' + j, 
                                    target_size = (160, 160))
        img = image.img_to_array(img)
        img = preprocess_input(img)
        X.append(img)
        y.append(i)

### Loading & Transforming our Test Data to a Numpy Array

In [6]:
for i in os.listdir('./dataset/test_set'):
    true_class = i
    for j in os.listdir('./dataset/test_set/' + i):
        img = image.load_img('./dataset/test_set/' + i + '/' + j, 
                                    target_size = (160, 160))
        img = image.img_to_array(img)
        img = preprocess_input(img)
        X.append(img)
        y.append(i)

In [7]:
X = np.array(X)
y = np.array(y)

In [8]:
print(X.shape)
print(y.shape)

(7496, 160, 160, 3)
(7496,)


In [9]:
le = LabelEncoder()
le.fit(y)
y = le.transform(y)

In [10]:
y = to_categorical(y)

### Running 10-Fold Cross Validation on MobileNet Model

In [12]:
kfold = StratifiedKFold(n_splits = 10, shuffle = True, random_state = 42)
cvscores = list()
confusionM = np.zeros((8, 8))

k = 10

for i, (train, test) in enumerate(kfold.split(X, np.argmax(y, axis = 1))):
    print("\n")
    print(f'Fold Run {i + 1} / {k}')
    
    #Create the Model
    base_model = MobileNet(input_shape = (160, 160, 3), weights = 'imagenet', include_top = False)

    for layer in base_model.layers:
        layer.trainable = False

    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(512, activation = 'relu')(x)
    x = Dense(256, activation = 'relu')(x) 
    x = Dense(128, activation = 'relu')(x)
    x = Dense(64, activation = 'relu')(x)
    preds = Dense(8, activation = 'softmax')(x)
    
    mobilenet_model = Model(inputs = base_model.input, outputs = preds)
    
    mobilenet_model.compile(optimizer = Adam(lr = 0.0001, beta_1 = 0.9, beta_2 = 0.999), loss = 'categorical_crossentropy', metrics = ['accuracy'])
    
    mobilenet_model.fit(X[train], y[train], epochs = 10, batch_size = 32)
    
    scores = mobilenet_model.evaluate(X[test], y[test], verbose = 0)
    predictions = np.argmax(mobilenet_model.predict(X[test]), axis = 1)
    
    runCM = confusion_matrix(np.argmax(y[test], axis = 1), predictions)  
    print("\nConfusion Matrix for Run " + str(i + 1) + ":")
    print(runCM)
    
    confusionM = confusionM + runCM
    
    print("Accuracy : %.2f%%" % (scores[1] * 100))
    cvscores.append(scores[1] * 100)



Fold Run 1 / 10
Epoch 1/10
6746/6746 [==============================] - 84s 12ms/step - loss: 0.8212 - accuracy: 0.7554
Epoch 2/10
6746/6746 [==============================] - 82s 12ms/step - loss: 0.3260 - accuracy: 0.8976
Epoch 3/10
6746/6746 [==============================] - 82s 12ms/step - loss: 0.2590 - accuracy: 0.9180
Epoch 4/10
6746/6746 [==============================] - 82s 12ms/step - loss: 0.2074 - accuracy: 0.9331
Epoch 5/10
6746/6746 [==============================] - 82s 12ms/step - loss: 0.1784 - accuracy: 0.9417
Epoch 6/10
6746/6746 [==============================] - 82s 12ms/step - loss: 0.1449 - accuracy: 0.9546
Epoch 7/10
6746/6746 [==============================] - 81s 12ms/step - loss: 0.1291 - accuracy: 0.9551
Epoch 8/10
6746/6746 [==============================] - 82s 12ms/step - loss: 0.0996 - accuracy: 0.9690
Epoch 9/10
6746/6746 [==============================] - 81s 12ms/step - loss: 0.0957 - accuracy: 0.9695
Epoch 10/10
6746/6746 [=======================

In [19]:
print("Average Accuracy with 10 Fold Cross Validation = %.2f%%" %(np.mean(cvscores)))

Average Accuracy with 10 Fold Cross Validation = 91.05%


In [31]:
cfm_n = np.flipud(confusionM)
cfm_n = cfm_n.astype('float') / cfm_n.sum(axis=1)[:, np.newaxis]

z = cfm_n
z = np.round(z, 2)
print(np.round(z, 2))
x = ['animals', 'art', 'automobiles', 'fashion', 'food', 'houses', 'technology', 'travel']
y = ['travel', 'technology', 'houses', 'food', 'fashion', 'automobiles', 'art', 'animals']
fig = ff.create_annotated_heatmap(z, x = x, y = y, annotation_text = z, colorscale = 'deep')
fig.update_layout(width = 800, height = 800, xaxis_showgrid = False, yaxis_showgrid = False, template = 'none')
fig.update_layout(title = {'text': "10-Fold Cross Validation Model",
                           'y':0.95,
                           'x':0.47,
                           'xanchor': 'center',
                           'yanchor': 'top'}, 
                  xaxis_title = 'Predicted Labels',
                  yaxis_title = 'True Labels')
fig['layout']['xaxis']['side'] = 'bottom'
fig.show()

[[0.01 0.02 0.   0.01 0.   0.11 0.01 0.84]
 [0.01 0.04 0.03 0.01 0.   0.03 0.88 0.01]
 [0.   0.01 0.   0.   0.   0.96 0.01 0.02]
 [0.   0.02 0.   0.   0.97 0.   0.01 0.  ]
 [0.   0.01 0.   0.97 0.   0.   0.01 0.  ]
 [0.   0.   0.98 0.   0.   0.   0.01 0.  ]
 [0.01 0.81 0.01 0.01 0.03 0.03 0.05 0.04]
 [0.88 0.04 0.01 0.01 0.02 0.01 0.02 0.02]]
